In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi
!cat /proc/cpuinfo
!rm -rf sample_data

!git clone --recursive https://github.com/Microsoft/LightGBM.git
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Tue Apr 20 03:43:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!mkdir data
# !cp /content/drive/MyDrive/Data-x/data/0403/*.csv /content/data/
!cp /content/drive/MyDrive/Data-x/data/filtered/*.csv /content/data/

In [ ]:
%cd data
!for file in *; do mv "${file}" "${file/_filtered/}"; done

/content/data


In [ ]:
%cd /content

/content


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager


N_SPLITS = 10
SEED = 42

@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]





set_seed(SEED)

feature_dir = './*/'
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
print(train_files)
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))

#parameters for the three LightGBM models
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.04,
              'num_leaves': 90,
              'device': 'gpu',
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 4,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.04,
                'num_leaves': 90,
                'device': 'gpu',
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 2,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }

['./data/5cd56b5ae2acfd2d33b58544_train.csv', './data/5cd56b5ae2acfd2d33b58546_train.csv', './data/5cd56b5ae2acfd2d33b58548_train.csv', './data/5cd56b5ae2acfd2d33b58549_train.csv', './data/5cd56b5ae2acfd2d33b5854a_train.csv', './data/5d27075f03f801723c2e360f_train.csv', './data/5d27096c03f801723c31e5e0_train.csv', './data/5d27097f03f801723c320d97_train.csv', './data/5d27099f03f801723c32511d_train.csv', './data/5d2709a003f801723c3251bf_train.csv', './data/5d2709b303f801723c327472_train.csv', './data/5d2709bb03f801723c32852c_train.csv', './data/5d2709c303f801723c3299ee_train.csv', './data/5d2709d403f801723c32bd39_train.csv']


7

# Calculate all data

In [ ]:
from sklearn.metrics import mean_squared_error

score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    # oof_x, oof_y, oof_f is the np.zeros to store X, Y and Floor result
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))
    print("Starting to train: ", os.path.basename(file))
    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-6])):
        X_train = data.iloc[trn_idx, :-6] # All wifi signal info
        y_trainx = data.iloc[trn_idx, -6] # the location X to be predicted
        y_trainy = data.iloc[trn_idx, -5] # the location Y to be predicted
        y_trainf = data.iloc[trn_idx, -4] # the location's floor

        X_valid = data.iloc[val_idx, :-6] # All wifi signal info in KFold validation dataset
        y_validx = data.iloc[val_idx, -6] # the location X to be predicted in KFold validation dataset
        y_validy = data.iloc[val_idx, -5] # the location Y to be predicted in KFold validation dataset
        y_validf = data.iloc[val_idx, -4] # the location's floor in KFold validation dataset

        modelx = lgb.LGBMRegressor(**lgb_params) #Model selected, which is LightGBM
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 1, used to predit X location only

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 2, used to predit Y location only

        modelf = lgb.LGBMClassifier(**lgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=False,
                       early_stopping_rounds=20
                       ) #Model 3, used to predit Floor location only

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        preds_x += modelx.predict(test_data.iloc[:, :-6]) / N_SPLITS 
        #Average the result of each folds, divide every result by n_splits, and add them together
        preds_y += modely.predict(test_data.iloc[:, :-6]) / N_SPLITS
        #Average the result of each folds, divide every result by n_splits, and add them together
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-6]).astype(int)
        #The floor is dealt differently,   

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        # the error in current KFold
        print(f"fold {fold}: mean position error {score}")


    # Evaluate mean position error of each file
    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -6].to_numpy(), data.iloc[:, -5].to_numpy(), data.iloc[:, -4].to_numpy())
    #Calculate the score/error
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    #The floor is dealt differently, vote to choose the most appeared floor number in all folds
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    ######
    #Stack all the predictions in one testing file together, and transpose to vertical
    predictions.append(test_preds)
    # print(test_preds)

    column_num = len(test_data.columns)
    test_data = test_data.reset_index()

    test_data.insert(column_num, 'pred_x', test_preds[1], True)
    test_data.insert(column_num+1, 'pred_y', test_preds[2], True)
    test_data.insert(column_num+2, 'pred_f', test_preds[0], True) 

    building_name = file[7:-10] + '.csv'

    test_data.to_csv(building_name)

print("*+"*40)
print(sum(oof)/len(oof))
print("*+"*40)

Starting to train:  5cd56b5ae2acfd2d33b58544_train.csv


[1.6GB(+0.0GB): 0.352sec] fit X
[1.6GB(+0.0GB): 0.519sec] fit Y
[1.6GB(+0.0GB): 1.630sec] fit F


fold 0: mean position error 13.040269271935818


[1.6GB(+0.0GB): 0.519sec] fit X
[1.6GB(+0.0GB): 0.662sec] fit Y
[1.6GB(+0.0GB): 0.961sec] fit F


fold 1: mean position error 15.231453072024527


...

fold 7: mean position error 5.937653574346606


[1.7GB(+0.0GB): 8.843sec] fit X
[1.7GB(+0.0GB): 5.225sec] fit Y
[1.7GB(+0.0GB): 4.078sec] fit F


fold 8: mean position error 5.925934298840672


[1.7GB(+0.0GB): 6.569sec] fit X
[1.7GB(+0.0GB): 5.674sec] fit Y
[1.7GB(+0.0GB): 3.927sec] fit F


fold 9: mean position error 5.916382263391724
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #13, shape=(2670, 440), name=5d2709d403f801723c32bd39_train.csv
mean position error 5.7451372152300175
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
10.900277790022207
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [ ]:
!cp *.csv 'drive/MyDrive/Data-x/predicted result filtered data/'